## Another variant of transfer learning

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#import cv2
import PIL.Image as Image

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
import tensorflow_hub as hub

print(f"Tensorflow Version {tf.__version__}, Keras Vesion: {keras.__version__}")

In [ ]:
IMG_SIZE = (224, 224)

l1 = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMG_SIZE+(3,))
classifier = tf.keras.Sequential([ l1 ])

In [ ]:
tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
image_labels = []
with open("ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()
print( f'''
Imagenet has {len(image_labels)} - labels.
First 5 labels are:  {image_labels[:5]}

''')

In [ ]:
gold_fish = Image.open("goldfish.jpg").resize(IMG_SIZE)
gold_fish

In [ ]:
gold_fish = np.array(gold_fish)/255.0
result = classifier.predict(gold_fish[np.newaxis, ...])

predicted_label_index = np.argmax(result)
pl = f'Predicted label is: {image_labels[predicted_label_index]}'

result.shape, predicted_label_index, pl, gold_fish.shape

<h3 style='color:purple'>Load flowers dataset</h3>

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

In [ ]:
print(data_dir)

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
roses[:5]

In [ ]:
import PIL
PIL.Image.open(str(roses[1]))

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

<h3 style='color:purple'>Read flowers images from disk into numpy array using opencv</h3>

In [ ]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

In [ ]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}

In [ ]:
flowers_images_dict['roses'][:5]

In [ ]:
str(flowers_images_dict['roses'][0])

In [ ]:
import cv2
img = cv2.imread(str(flowers_images_dict['roses'][0]))

In [ ]:
img.shape

In [ ]:
cv2.resize(img,(224,224)).shape

In [ ]:
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
X = np.array(X)
y = np.array(y)

<h3 style='color:purple'>Train test split</h3>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

<h3 style='color:purple'>Preprocessing: scale images</h3>

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

**Make prediction using pre-trained model on new flowers dataset**

In [ ]:
X[0].shape

In [ ]:
IMG_SIZE+(3,)

In [ ]:
x0_resized = cv2.resize(X[0], IMG_SIZE)
x1_resized = cv2.resize(X[1], IMG_SIZE)
x2_resized = cv2.resize(X[2], IMG_SIZE)

In [ ]:
plt.axis('off')
plt.imshow(X[0])

In [ ]:
plt.axis('off')
plt.imshow(X[1])

In [ ]:
plt.axis('off')
plt.imshow(X[2])

In [ ]:
predicted = classifier.predict(np.array([x0_resized, x1_resized, x2_resized]))
predicted = np.argmax(predicted, axis=1)
predicted

In [ ]:
image_labels[795]

<h3 style='color:purple'>Now take pre-trained model and retrain it using flowers images</h3>

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

In [ ]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
model.fit(X_train_scaled, y_train, epochs=3, validation_split=0.2, batch_size=32)

In [ ]:
model.evaluate(X_test_scaled,y_test)

# Homework

Now you have used a different method to train and use the model. Could you load the dataset and try to use your model and see the evaluation results?

(a) what metrics you see
(b) why 
(c) could you think of a way to modify the model to handle the dataset
(d) what did you observe and how can you use this knowledge in future to developing new models

In [ ]:
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(  "tf_flowers",
                        split=["train[:70%]", "train[:30%]"], ## Train test split
                        batch_size=-1,
                        as_supervised=True) 

num_classes = 5
train_ds1 = tf.image.resize(train_ds, IMG_SIZE)
test_ds1  = tf.image.resize(test_ds, IMG_SIZE)
train_ds2 = preprocess_input(train_ds1)/255.0
test_ds2  = preprocess_input(test_ds1) /255.0

## Transforming labels to correct format
train_labels1 = to_categorical(train_labels, num_classes=num_classes)
test_labels1  = to_categorical(test_labels, num_classes=num_classes)

model.evaluate(test_ds2, test_labels)


In [ ]:
#y_test, test_labels